Author: Simon Ng <br/> Created: 2024-12-17 <br/> Last edited: 2025-02-10 <br/><br/>
This code is the 2nd step in a project to 3D print a model of the Engadin valley in Switzerland as a Christmas gift to a friend who is skiing the Engadin Ski Marathon this year! The project has 5 steps: <br/>
1. Select area of interest from an open source Swiss elevation dataset (https://www.swisstopo.admin.ch/en/height-model-swissalti3d). I used "selection by polygon" at 2.0m resolution to obtain a csv file with links to tiles covering the entire Engadin Ski Marathon and surrounding mountains. <br/>
2. Merge linked GeoTiff tiles in exported csv file into one file (<<< that's what this code does!) <br/>
3. Add geospatial features in QGIS, ArcGIS, etc such as trails, cities, etc. Note that additional features need to be represented as elevation data merged with the exported GeoTiff so they will be part of the eventual 3D print! Also a good time to resample the pixel resolution. 2 meter resolution won't show up for a reasonably scaled 3D print but WILL make processing times horrific. Something like 30 meter resolution is better. This is also the time to re-mask your area of interest since the swisstopo site exports blocky edges on diagonals. <br/>
4. Convert edited GeoTiff to STL mesh for slicing and 3D printing
5. Slice and 3D print it!

In [1]:
# Import packages
import pandas as pd
import xarray
import rioxarray # to open and download remote raster data
from rioxarray.merge import merge_arrays
import numpy as np
import warnings
warnings.filterwarnings('ignore') # have a warning "UserWarning: angle from rectified to skew grid parameter lost in conversion to CF". Doesn't appear to affect final GeoTiff, so ignoring warning

C:\Users\ngsim\UserApps_SN\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Read file with GeoTiff links (data from https://www.swisstopo.admin.ch/en/height-model-swissalti3d)
TifCSV = r"C:\Users\ngsim\Documents\Engadin\ch.swisstopo.swissalti3d-kxgyFAfD.csv"
Links = pd.read_csv(TifCSV, header = None)
Links

,0
0,https://data.geo.admin.ch/ch.swisstopo.swissal...
1,https://data.geo.admin.ch/ch.swisstopo.swissal...
2,https://data.geo.admin.ch/ch.swisstopo.swissal...
3,https://data.geo.admin.ch/ch.swisstopo.swissal...
4,https://data.geo.admin.ch/ch.swisstopo.swissal...
...,...
650,https://data.geo.admin.ch/ch.swisstopo.swissal...
651,https://data.geo.admin.ch/ch.swisstopo.swissal...
652,https://data.geo.admin.ch/ch.swisstopo.swissal...
653,https://data.geo.admin.ch/ch.swisstopo.swissal...


In [3]:
def merge_tiles(Links, first_tile_index = 0, last_tile_index = len(Links), save_to = ""):
    '''
    merge_tiles opens all GeoTiff tiles linked in the csv and merges them into one GeoTiff
    Inputs: links to GeoTiff tiles as pandas dataframe, optional inputs for which tiles to merge (in case there are issues with processing speed). last_tile_index is non-inclusive
    Returns: merged GeoTiff of all tiles within indices provided
    '''
    
    # Create empty list to store tiles
    TileList = []
    
    # Open tiles and add them to list
    print(f"Adding tiles {first_tile_index}-{last_tile_index} to list...")
    
    for i, link in enumerate(Links[first_tile_index:last_tile_index][0]): # loop through tiles within first/last tile indices
        
        NewTile = rioxarray.open_rasterio(link) # retrieve new tile from https links
        TileList.append(NewTile) # add tile to list
        #print(f"Tile {i+first_tile_index} added to list")   # tiles add very fast. This seems like an unnecessary user update
        
    # Merge tiles
    print("Merging tiles...")
    merged_tiles = merge_arrays(TileList)
    print("Tiles merged")
    
    # Save merged GeoTiff (optional)
    if save_to != "":
        merged_tiles.rio.to_raster(save_to) # save geotiff
        print(f"Saved merged GeoTiff to {save_to}")
        
    return merged_tiles

In [4]:
# Use merge_tiles function to actually merge tiles
# Whether to just run merge_tiles once on all tiles and save the GeoTiff or run it in batches depends on processing time and how well GIS software handles the output file size

# Example merging all tiles at once and saving
#merged_tiles = merge_tiles(Links, save_to = r"C:\Users\ngsim\Documents\Engadin\250211_MergedTiles_655.tif")

# Example of merging the tiles in batches
merged_tiles1 = merge_tiles(Links, first_tile_index = 0, last_tile_index = 100, save_to = r"C:\Users\ngsim\Documents\Engadin\250211_MergedTiles_0_100.tif")  # only saving this one as a test with a small subset of data
#merged_tiles2 = merge_tiles(Links, first_tile_index = 100, last_tile_index = 200)
#merged_tiles3 = merge_tiles(Links, first_tile_index = 200)'''

Adding tiles 0-100 to list...
Merging tiles...
Tiles merged
Saved merged GeoTiff to C:\Users\ngsim\Documents\Engadin\250211_MergedTiles_0_100.tif


In [5]:
# Optional merge all the smaller merges into a big merge
#MergedTileList = [MergedTiles1, MergedTiles2, MergedTiles3]
#AllMergedTiles = merge_arrays(MergedTileList)
#AllMergedTiles.rio.to_raster(r"C:\\Users\\ngsim\\Downloads\\241217_AllMergedTiles.tif") # save geotiff